# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
# File to Load (Remember to Change These)
city_data_load = "WeatherPy.csv"
#student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
city_df = pd.read_csv(city_data_load)
#student_data = pd.read_csv(student_data_to_load)
city_df.dropna(inplace = True) 
city_df

,City,Country,Max Temp,Date,Cloudiness,Lat,Lng,Wind Speed,Humidity
0,Bluff,NZ,278.71,1619371975,79,-46.6000,168.3333,2.68,84
1,Miramar,US,305.93,1619371976,75,25.9873,-80.2323,4.63,51
2,Port Alfred,ZA,294.62,1619371976,98,-33.5906,26.8910,6.88,71
3,Belciugatele,RO,289.26,1619371977,70,44.4833,26.4333,0.45,40
4,Nemuro,JP,273.39,1619371969,37,43.3236,145.5750,9.05,68
...,...,...,...,...,...,...,...,...,...
544,Ulaanbaatar,MN,273.15,1619372128,75,47.9077,106.8832,4.00,80
545,Preobrazheniye,RU,275.24,1619372128,0,42.9019,133.9064,1.66,52
546,Tyrma,RU,271.78,1619372129,59,50.0833,132.1667,1.12,47
547,Conceição da Barra,BR,300.41,1619372129,40,-18.5933,-39.7322,4.18,71


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
# Store latitude and longitude in locations
locations = city_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = city_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
city1 = city_df.loc[(city_df["Max Temp"] < 80) & (city_df["Max Temp"] > 70)]
city1

,City,Country,Max Temp,Date,Cloudiness,Lat,Lng,Wind Speed,Humidity


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [ ]:
# Add marker layer ontop of heat map


# Display figure
